This notebook's goal is to make the user count for a certain group be reflected in the area for the treemap

In [1]:
from utils import mount_src
mount_src()
import polars as pl
import hypothesis_row as hr
import treemap as tm

DATA = "../data/movie_lens.csv"
USER_COUNT = "../data/user_counts/movie_lens.csv"

In [2]:
df = pl.read_csv(DATA)
users = pl.read_csv(USER_COUNT)

df

algorithm,step,g_in,h,G_out,coverage,diversity,normalized_diversity
str,i64,str,str,str,f64,f64,f64
"""Greedy-HEP""",0,"""All users""","""Mean is greater than 3.5""","""runtime_minutes:Long, location…",1.0,2.99255,0.997517
"""Greedy-HEP""",1,"""runtime_minutes:Long""","""Mean is greater than 3.5""","""location:HI|runtime_minutes:Lo…",0.999503,2.992712,0.997571
"""Greedy-HEP""",2,"""genre:Drama|runtime_minutes:Lo…","""Mean is greater than 3.5""","""genre:Drama|runtime_minutes:Lo…",0.989399,2.992968,0.997656
"""Greedy-HEP""",3,"""genre:Drama|runtime_minutes:Lo…","""Mean is greater than 3.5""","""gender:F|genre:Drama|runtime_m…",1.0,2.993106,0.997702
"""Greedy-HEP""",4,"""gender:M|genre:Drama|runtime_m…","""Mean is greater than 3.5""","""age:25-34|gender:M|genre:Drama…",0.740014,3.0,1.0
…,…,…,…,…,…,…,…
"""RL-HEP""",30,"""age:50-55|location:MN""","""Mean is greater than 3.5""","""age:50-55|genre:Action|locatio…",1.0,1.158901,0.3863
"""RL-HEP""",31,"""age:50-55|genre:Action|locatio…","""Mean is greater than 3.5""","""age:50-55|gender:M|genre:Actio…",1.0,0.322581,0.322581
"""RL-HEP""",32,"""genre:Drama""","""Mean is greater than 3.5""","""genre:Drama|location:NE, genre…",1.0,2.993043,0.997681


In [3]:
hs = hr.HypothesisRow.df_to_list(df, users)
hs[0].get_parent_child_df()

parent,child,label,hypothesis,count
str,str,str,str,i64
"""""","""all_users:all""","""all""","""Mean is greater than 3.5""",6040
"""all_users:all""","""runtime_minutes:Long""","""Long""","""Mean is greater than 3.5""",6040
"""all_users:all""","""location:AL""","""AL""","""Mean is greater than 3.5""",23
"""all_users:all""","""location:RI""","""RI""","""Mean is greater than 3.5""",22


In [4]:
tm.make_treemap_from_range(hs, 0, 1)